# Keras-based Machine Learning Pipeline

This is a pipeline for doing data analsis using the Keras package and Tensorflow.

**Steps:**
1. Load SITL reports
2. Load MMS data
3. Data preprocessing
4. Train LSTM model

Introduction to LSTMs and Keras: https://adventuresinmachinelearning.com/keras-lstm-tutorial/

Can base this on an example from Machine Learning Mastery: https://machinelearningmastery.com/how-to-develop-rnn-models-for-human-activity-recognition-time-series-classification/

In [1]:
import os
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from matplotlib import pyplot
from matplotlib.dates import num2date, date2num
import random

# Machine learning
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import retrieve_sitl
import retrieve_mms

## 1. Load SITL data

Load parsed reports in. Following would be useful:
- Reports reduced to "BBF" / "DF" / some other interesting signals
- Starttime and endtime and duration provided
- If the report was for multiple events or single event or not
- FOM value

In [2]:
# Read the reports in 2016 with DFs:
pickle_path = "../pydata/DF_2016_reports_parsed_df.p"   # pickle created in example_sitl_read.ipynb
reports_df_2016 = pd.read_pickle(pickle_path)
reports_df_2016

,FOM,ID,Discussion,Starttime,Endtime,Day,FOM_float,Duration,Discussion_contains_BBF,Discussion_contains_DF,Discussion_contains_uncertainty
124,25.0,icohen(EVA),"Dipolarization features, particle boundary an...",2016-04-13 00:37:54,2016-04-13 01:04:54,2016-04-13,25.0,0 days 00:27:00,False,True,False
298,40.0,jburch(EVA),Possible dipolarization front,2016-04-24 12:13:14,2016-04-24 16:18:24,2016-04-24,40.0,0 days 04:05:10,False,True,False
312,20.0,jburch(EVA),Possible dipolarization front,2016-04-26 10:22:34,2016-04-26 10:26:34,2016-04-26,20.0,0 days 00:04:00,False,True,False
327,40.0,jburch(EVA),Possible dipolarization event,2016-04-27 15:14:34,2016-04-27 15:45:04,2016-04-27,40.0,0 days 00:30:30,False,True,False
341,40.0,jburch(EVA),"Possible dipolarization front, 400 km/s Sunwa...",2016-04-29 10:18:24,2016-04-29 10:23:14,2016-04-29,40.0,0 days 00:04:50,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...
1482,10.0,ocontel(EVA),Small dipolarization with dawnward flow (Vy=...,2016-09-11 20:44:04,2016-09-11 20:50:14,2016-09-11,10.0,0 days 00:06:10,False,True,False
1487,20.0,ocontel(EVA),DF with Vy=-300 km/s,2016-09-11 22:03:04,2016-09-11 22:08:54,2016-09-11,20.0,0 days 00:05:50,False,True,False
1596,10.0,ocontel(EVA),Dawnward flow Vy=-150 km/s (DF like) with LF...,2016-09-16 20:20:44,2016-09-16 20:27:24,2016-09-16,10.0,0 days 00:06:40,False,True,False
1655,15.0,hhiroshi(EVA),weak earthward & dawnward flow with possible ...,2016-09-20 00:56:04,2016-09-20 01:05:34,2016-09-20,15.0,0 days 00:09:30,False,True,True


In [3]:
# Here's an example event, but ideally we would have a list of these events:
#events = reports_df_2016   # correct version
events = reports_df_2016[reports_df_2016.Day.str.contains('2016-09-11')]
# --------------------------------------------------------------
# USE ALL EVENTS WHEN ALL DATA AVAILABLE! I only used one example day.
# --------------------------------------------------------------
events

,FOM,ID,Discussion,Starttime,Endtime,Day,FOM_float,Duration,Discussion_contains_BBF,Discussion_contains_DF,Discussion_contains_uncertainty
1479,10.0,ocontel(EVA),"Small dipolarization, low frequency E waves ...",2016-09-11 18:35:24,2016-09-11 18:40:04,2016-09-11,10.0,0 days 00:04:40,False,True,False
1481,10.0,ocontel(EVA),Small dipolarization with low frequency E wa...,2016-09-11 20:36:24,2016-09-11 20:42:14,2016-09-11,10.0,0 days 00:05:50,False,True,False
1482,10.0,ocontel(EVA),Small dipolarization with dawnward flow (Vy=...,2016-09-11 20:44:04,2016-09-11 20:50:14,2016-09-11,10.0,0 days 00:06:10,False,True,False
1487,20.0,ocontel(EVA),DF with Vy=-300 km/s,2016-09-11 22:03:04,2016-09-11 22:08:54,2016-09-11,20.0,0 days 00:05:50,False,True,False


## 2. Load MMS data

Load in raw data for processing.

Load both magnetic (FGM) and plasma/ion data (FPI, not yet implemented). Both will be useful to feed into a machine learning algorithm.

In [4]:
# Read FGM data:
# ---------------------------------------
#starttime, endtime = datetime(2016,6,1), datetime(2017,1,1)   # these dates should be for the available data
# ---------------------------------------
starttime, endtime = datetime(2016,9,11), datetime(2016,9,12)
local_data_dir = '../pydata'
base_path = os.path.join(local_data_dir, 'mms1', 'fgm', 'srvy', 'l2')
data = retrieve_mms.get_fgm(base_path, starttime, endtime)
t_utc, B_x, B_y, B_z, Bt = data

Reading file at ../pydata/mms1/fgm/srvy/l2/2016/09/mms1_fgm_srvy_l2_20160911_v5.87.0.cdf


In [ ]:
# Also read FPI data:
base_path = os.path.join(local_data_dir, 'mms1', 'fpi', 'fast', 'l2', 'dis-moms')
data = retrieve_mms.get_fpi(base_path, starttime, endtime)
t_utc_fpi, speed_x, speed_y, speed_z, density = data

## 3. Data proprocessing

First steps...

1. Resample data to 1s? I don't think we need a higher resolution. Maybe somebody else knows better here.
2. Label data with classes, e.g. all data has class 0 to start with, data marked as BBF in reports is then labelled with class 1, then data marked as DF is labelled with class 2, etc. We can try to predict the event type contained in a short time series within the data.
3. May want to remove general trends so data is static. I think the signals we're looking for are only related to variations, not to overall increase/decrease in magnetic field values.

In [5]:
# Resample data:
# --------------
# (Iterating over datetime takes a while - this could be optimised)
t_sec = np.array([starttime + timedelta(seconds=n) for n in range(int((endtime-starttime).total_seconds()))])
t_sec_num = date2num(t_sec)
t_utc_num = date2num(t_utc)
# Interpolate to new time array:
B_x_sec = np.interp(t_sec_num, t_utc_num, B_x)
B_y_sec = np.interp(t_sec_num, t_utc_num, B_y)
B_z_sec = np.interp(t_sec_num, t_utc_num, B_z)
Bt_sec = np.interp(t_sec_num, t_utc_num, Bt)
# --------------------------------------------------------------
# Could add in gradient here! Detrending would be more complicated because each slice (below) would have to be detrended
# --------------------------------------------------------------
# FPI data:     # uncomment this when FPI data is available!
#t_utc_fpi_num = date2num(t_utc_fpi)
#speedx_sec = np.interp(t_sec_num, t_utc_fpi_num, speed_x)   # can also try with y and z
#density_sec = np.interp(t_sec_num, t_utc_fpi_num, density)

# Combine all features (input data) into array:
all_features = np.hstack(( B_x_sec.reshape(-1,1), 
                           B_y_sec.reshape(-1,1), 
                           B_z_sec.reshape(-1,1),
                           Bt_sec.reshape(-1,1)),
                           #speedx_sec.reshape(-1,1),
                           #density_sec.reshape(-1,1)     # uncomment this when FPI data is available!
                        )

In [6]:
# Create arrays with data classes:
# --------------------------------
# 1 is the label in the data for "contains DF"!
t_classes = np.zeros(len(t_sec))
for index, event in events.iterrows():
    t_classes[np.logical_and(t_sec >= event.Starttime, t_sec < event.Endtime)] = 1
print("Values in array:", np.unique(t_classes))
print("DFs: {:.1f} % of data set".format(np.count_nonzero(t_classes)/len(t_classes)*100.))

Values in array: [0. 1.]
DFs: 1.6 % of data set


### Steps in preparation for machine learning

Combine all MMC variables into one array, and then...

1. Split MMS data into 30-minute time slices (samples).
2. Give each section a class if it contains an interesting signal. Use keras.utils.to_categorical() on this.
3. Scale input data for LSTM (sklearn.MixMaxScaler).
4. Make a train-test split of all data.
5. Input data (X) should be n-dimensional and contain magnetic field variations and ion data. **What else can we add to this?** Output data (y) should be 2-dimensional and contain one class (int value, 0 or 1) per time slice. It's a good place to start.

In [7]:
# Data preprocessing variables
# ----------------------------
# Minutes in slice determines the size of the samples given to the net:
mins_in_slice = 5 # 30
# Batch size determines the number of samples given to the net for one iteration:
batch_size = 4 # try to keep this as a power of 2
# Epochs is the number of times the neural net / LSTM iterates to find the best. 
# A higher number usually means more accurate, but put it too high and you overtrain.
epochs = 100

In [8]:
# Define X - the model input
# --------------------------
# Scale data to the range (0,1), NN won't be happy with unscaled data
scaler_X = MinMaxScaler()
scaler_X.fit(all_features)
all_features_scaled = scaler_X.transform(all_features)
# X contains the magnetic field measurements and (soon) the plasma instrument data
all_features_split = np.array_split(all_features_scaled, 86400/(60*mins_in_slice))
X_all = np.array(all_features_split)

# Define y - the desired output
# -----------------------------
# In this case y is binary: 0 (= no DF) or 1 (= contains DF)
t_classes_split = np.array_split(t_classes, 86400/(60*mins_in_slice))
y_all = np.array([int(max(x)) for x in t_classes_split]).reshape(-1,1)

# Reduce samples so that number of samples with/without DFs are balanced
# ----------------------------------------------------------------------
# Find where DFs are:
DF_slices = np.where(y_all==1)[0]
all_slices = [x for x in list(range(len(y_all))) if x not in DF_slices]
# Find where DFs aren't and get equal number of samples:
noDF_slices = random.sample(all_slices, len(DF_slices))
X_DFs = X_all[DF_slices]
X_noDFs = X_all[noDF_slices]
X = np.vstack((X_DFs, X_noDFs))
y_DFs = y_all[DF_slices]
y_noDFs = y_all[noDF_slices]
y = np.vstack((y_DFs, y_noDFs))

# Make y into categorical:
y = to_categorical(y)

# Output sizes:
print("Number of samples:\t\t", X.shape[0])
print("Number of timesteps:\t\t", X.shape[1])
print("Number of features in X:\t", X.shape[2])
print("Number of categories in y:\t", y.shape[1])
print("Shape of X = {}, y = {}".format(X.shape, y.shape))

Number of samples:		 16
Number of timesteps:		 300
Number of features in X:	 4
Number of categories in y:	 2
Shape of X = (16, 300, 4), y = (16, 2)


In [14]:
# Split data into training and testing:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

## 4. Train LSTM model

Take this as starting example: https://machinelearningmastery.com/how-to-develop-rnn-models-for-human-activity-recognition-time-series-classification/

Ideas on how to improve the accuracy:
- Change the LSTM format. Remove the dropout, change it, add a special metric, etc.
- Use more epochs. Find a cut-off point before it starts overtraining (which is when the training accuracy is much higher than the testing accuracy, which shouldn't happen).
- Change the batch_size.
- Change up the mins_in_slice variable. Is it better with shorter or longer time slices? It may be better with longer time slices because some reports include DFs with comments on other signals, leading to some time slices being labelled as DFs but having no relevant signal. These will contaminate the dataset.
- Add other features into the input array X, e.g. detrended data, maybe a derivative, or combinations of different variables.
- Remove some features - too many can also bad.
- Only use data with higher FOM values.

In [10]:
# Define basic machine learning model:
def evaluate_model(trainX, trainy, testX, testy, epochs=15, batch_size=64):
    verbose = 0
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, train_accuracy = model.evaluate(trainX, trainy, batch_size=batch_size, verbose=0)
    _, test_accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return train_accuracy, test_accuracy

In [15]:
score_train, score_test = evaluate_model(X_train, y_train, X_test, y_test, batch_size=batch_size, epochs=epochs)
print("{:.1f}% correct in training".format(score_train*100.))
print("{:.1f}% correct in testing".format(score_test*100.))

58.3% correct in training
25.0% correct in testing


## 5. Apply trained model to new times

Once model is trained, can we use it to find similar signals in data outside what we've trained it on? Would be cool.

In [ ]:
# would be nice...